# Mixture of Linear Regressions

In this note, we will solve the mixture of linear regressions problem using the moment method.

$\newcommand{\E}{\mathbb{E}}$
$\renewcommand{\Re}{\mathbb{R}}$
$\newcommand{\oft}[1]{{^{(#1)}}}$
$\newcommand{\oftt}[2]{{^{(#1)#2}}}$

In [136]:
import util
from util import *
from models import LinearRegressionsMixture
import numpy as np
np.set_printoptions(precision=3, suppress=True)
from IPython.display import display, Markdown, Math
from operator import mul
sp.init_printing()

reload(util)

<module 'util' from 'util.pyc'>

Let $w_1, ..., w_K \in \Re^D$ be a set of $K$ regression coefficients. Let $x_1, ..., x_N \in \Re^D$ be a given set of data points and let $y_n = \sum_{k} \delta_k w_k^\top x_n + \epsilon$ be observed responses. Our objective is to recover $(w_k)$.

## Toy Example

Let's construct a toy example for the rest of this document.

In [112]:
K, D = 2, 2
np.random.seed(0)
model = LinearRegressionsMixture.generate("tmp.dat", K, D)
print("True parameters:")
print(model.weights)
print(model.betas)

True parameters:
[ 0.597  0.403]
[[ 1.868 -0.977]
 [ 0.95  -0.151]]


## Noiseless, Infinite Data Setting

In this scenario, assume that $\epsilon = 0$ and that we observe expected moments of the data. Consider the moments:
\begin{align}
\E[x^\alpha y^b] 
&= \sum_k \pi_k \E[x^\alpha y^b | \delta_k = 1] \\
&= \sum_k \pi_k \E[x^\alpha [w_k^\top x]^b] \\
&= \sum_k \pi_k \E[x^\alpha \sum_{\beta \in p(b)} w_k^\beta x^\beta] \\
&= \sum_{\beta \in p(b)} \E_\pi[w^\beta] \E[x^{\alpha + \beta}],
\end{align}
where $p(b)$ are the $d$-partitions of $b$. Note that $\E[x^{\alpha + \beta}]$ are observable quantities, and thus this simply represents a mixture over the polynomials with terms $w_k^\beta$.

In [113]:
def evaluate_mixture(ws, pis, beta):
    """
    Compute E_\pi[w^beta]
    """
    return sum(pi * util.monomial(w, beta) for w, pi in zip(ws.T, pis))
    
def compute_exact_y_moments(ws, pis, moments_x, alpha, b):
    """
    Compute the exact moments E[x^a y^b] using coefficients ws, pis and moments_x.
    """
    D, K = ws.shape
    ret = 0.
    for beta in partitions(D, b):
        ret += evaluate_mixture(ws, pis, beta) * moments[tuple_add(alpha, beta)]
    return ret

# compute_exact_y_moments(model.betas, model.weights, moments, (0,0), 1)

In [114]:
import sympy as sp

R, _ = sp.xring(['w%d'%d for d in xrange(D)], sp.RR, sp.grevlex)

def describe_moment_polynomial(R, moments_x, moment_y, alpha, b):
    """
    Computes the moment polynomial for E[x^alpha, y^b]
    """
    D = len(R.symbols)
    w = R.symbols
    expr = -moment_y
    for beta in partitions(D, b):
        expr += util.monomial(w, beta) * moments[tuple_add(alpha, beta)]
    return expr

In [151]:
# Example 
def double_factorial(n): 
    return reduce(mul, xrange(n, 0, -2)) if n > 0 else 1
def gaussian_moments(sigma, d):
    """
    E[x^d] where x is standard gaussian with sigma
    """
    if d == 0: return 1
    elif d % 2 == 0: return double_factorial(d-1) * sigma**d
    else: return 0
    
sigma = 1.0
max_moment = 8
#moments = {i : 1 for i in dominated_elements((3 for _ in xrange(D)))}
moments = {i : reduce(mul, [gaussian_moments(sigma, ii) for ii in i])
           for i in dominated_elements((max_moment for _ in xrange(D)))}
display(moments)
alpha, b = (0,1), 1
m = compute_exact_y_moments(model.betas, model.weights, moments, alpha, b)
display("exact moment", m)
display("polynomial", describe_moment_polynomial(R, moments, m, alpha, b))

'exact moment'

'polynomial'

With this machinery, we can compute the moment polynomials required for the moment method magic!

In [153]:
import MomentMatrix as mm
reload(mm)

# Fill in the constraints.
constrs = []
for b in [1,5]:
    for alpha in util.dominated_elements((3,3)):
        m = compute_exact_y_moments(model.betas, model.weights, moments, alpha, b)
        print "moments for b = %d, alpha = %s = %f" % (b, alpha, m)
        constrs.append( describe_moment_polynomial(R, moments, m, alpha, b) )
display(constrs)
M, sol = mm.solve_moments_with_constraints(R.symbols, constrs, 3)

moments for b = 1, alpha = (3, 3) = 0.000000
moments for b = 1, alpha = (3, 2) = 2.164614
moments for b = 1, alpha = (3, 1) = 0.000000
moments for b = 1, alpha = (3, 0) = 2.164614
moments for b = 1, alpha = (2, 3) = 1.519139
moments for b = 1, alpha = (2, 2) = 0.000000
moments for b = 1, alpha = (2, 1) = 0.506380
moments for b = 1, alpha = (2, 0) = 0.000000
moments for b = 1, alpha = (1, 3) = 0.000000
moments for b = 1, alpha = (1, 2) = 0.721538
moments for b = 1, alpha = (1, 1) = 0.000000
moments for b = 1, alpha = (1, 0) = 0.721538
moments for b = 1, alpha = (0, 3) = 1.519139
moments for b = 1, alpha = (0, 2) = 0.000000
moments for b = 1, alpha = (0, 1) = 0.506380
moments for b = 1, alpha = (0, 0) = 0.000000
moments for b = 5, alpha = (3, 3) = 0.000000
moments for b = 5, alpha = (3, 2) = 1585.241853
moments for b = 5, alpha = (3, 1) = 0.000000
moments for b = 5, alpha = (3, 0) = 1447.462500
moments for b = 5, alpha = (2, 3) = 436.924391
moments for b = 5, alpha = (2, 2) = 0.000000
mo

ValueError: Rank(A) < p or Rank([G; A]) < n